In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)


## Import the datasets

In [2]:
train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')

x_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
x_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
x_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')

x_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
x_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
x_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

x_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
x_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
x_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')

x_train_merged_a = pd.concat([x_train_observed_a,x_train_estimated_a])
x_train_merged_b = pd.concat([x_train_observed_b,x_train_estimated_b])
x_train_merged_c = pd.concat([x_train_observed_c,x_train_estimated_c])

## Merge x_train observed and x_train_estimated

In [3]:
x_train_a = pd.concat([x_train_observed_a,x_train_estimated_a])
x_train_b = pd.concat([x_train_observed_b,x_train_estimated_b])
x_train_c = pd.concat([x_train_observed_c,x_train_estimated_c])

## Data Preprocessing

1. Aggregate data from every 15 minutes to hourly intervals:
   - Method 1: Take the mean over all four 15minutes recording, resulting in hourly measurements.
   - Method 2: Create a separate column for each 15-minute value, effectively quadrupling the number of columns.
2. Handle missing values
   - Remove rows with NaN values in pv_measurement
   - Remove rows with timestamps that are not present in both x and y
3. 


### 1. Transform data to hourly

**Method 1: Take the mean over all four 15minutes recording, resulting in hourly measurements.**

In [4]:
def resample_to_hourly(x): 
    df = x.copy()
    df.set_index('date_forecast', inplace=True)
    
    # Aggregating by averaging over quartarly measurements
    df_hourly = df.resample('H').mean(numeric_only=False)
    df_hourly.reset_index(inplace=True)
    return df_hourly

x_train_a_hourly = resample_to_hourly(x_train_a)
x_train_b_hourly = resample_to_hourly(x_train_b)
x_train_c_hourly = resample_to_hourly(x_train_c)

x_test_a_hourly = resample_to_hourly(x_test_estimated_a)
x_test_b_hourly = resample_to_hourly(x_test_estimated_b)
x_test_c_hourly = resample_to_hourly(x_test_estimated_c)

In [5]:
#use only rows in test that are given in the test csv
test = pd.read_csv('test.csv')
pred_time_stamps = test['time'].unique()
x_test_a_hourly = x_test_a_hourly[x_test_a_hourly['date_forecast'].isin(pred_time_stamps)]
x_test_b_hourly = x_test_b_hourly[x_test_b_hourly['date_forecast'].isin(pred_time_stamps)]
x_test_c_hourly = x_test_c_hourly[x_test_c_hourly['date_forecast'].isin(pred_time_stamps)]


**Method 2: Create a separate column for each 15-minute value, quadrupling the number of columns.**

In [6]:
def resample_to_hourly_quarters(x, date_column='date_forecast', exclude_column='date_calc'):
    df = x.copy()
    # Ensure the date column is in datetime format and set as the index
    df[date_column] = pd.to_datetime(df[date_column])
    df.set_index(date_column, inplace=True)
    
    # Separate the column to exclude from the resampling
    excluded_data = df[[exclude_column]].resample('H').first()  # You can use 'first' or 'last' here
    
    # Remove the excluded column from df before pivoting
    df = df.drop(columns=[exclude_column])

    # Add a column for the 15-minute period within the hour
    df['quarter'] = df.index.minute // 15  # Use floor division to get the quarter number (0, 1, 2, 3)

    # Pivot the table. For each feature, create a new column for each 15-minute period.
    df_pivot = df.pivot_table(index=df.index.floor('H'),
                              columns='quarter',
                              aggfunc='first')  # We use 'first' because each quarter should be unique
    
    # Flatten the multi-level column index
    df_pivot.columns = ['{}_Q{}'.format(feature, quarter) for feature, quarter in df_pivot.columns]

    # Reset the index to be able to merge on the date_column
    df_pivot.reset_index(inplace=True)
    excluded_data.reset_index(inplace=True)

    # Merge back the excluded column
    df_hourly = pd.merge(excluded_data, df_pivot, on=date_column)

    return df_hourly

# Make sure to pass the column name that contains the datetime information
x_train_a_hourly2 = resample_to_hourly_quarters(x_train_a, date_column='date_forecast')
x_train_b_hourly2 = resample_to_hourly_quarters(x_train_b, date_column='date_forecast')
x_train_c_hourly2 = resample_to_hourly_quarters(x_train_c, date_column='date_forecast')

x_test_a_hourly2 = resample_to_hourly_quarters(x_test_estimated_a, date_column='date_forecast')
x_test_b_hourly2 = resample_to_hourly_quarters(x_test_estimated_b, date_column='date_forecast')
x_test_c_hourly2 = resample_to_hourly_quarters(x_test_estimated_c, date_column='date_forecast')

## 2. Handle constant pv measurments

In [7]:
def remove_constant_intervals(y_train, low_thresh, upp_thresh = 10**6):
    """
    Identify and remove intervals of constant PV readings that exceed a specified duration. 
    Constant readings may indicate sensor malfunctions or data logging issues.
    """
    
    df = y_train.copy()
    
    # Calculate the difference in production values
    df['diff'] = df['pv_measurement'].diff()

    # Identify where the difference is zero
    df['zero_diff'] = df['diff'].abs() < 1e-5

    # Identify groups of consecutive zero differences
    df['group'] = (df['zero_diff'] != df['zero_diff'].shift()).cumsum()

    # Filter out only the groups with consecutive zero differences
    constant_intervals = df[df['zero_diff']].groupby('group').agg(start=('time', 'min'), 
                                                                  end=('time', 'max'),
                                                                  duration=('time', 'size'))
    
    # Filter intervals based on the threshold
    interval_df_thresh = constant_intervals[(constant_intervals['duration'] > low_thresh) & (constant_intervals['duration'] <upp_thresh)]
    
    # Remove rows from the main dataframe that fall within these intervals
    for _, row in interval_df_thresh.iterrows():
        start_time, end_time = row['start'], row['end']
        df = df[(df['time'] < start_time) | (df['time'] > end_time)]
    
    # Drop the added columns used for calculations
    df.drop(columns=['diff', 'zero_diff', 'group'], inplace=True)
    
    return df

In [8]:
#Remove rows in groups of constant values, where duration of constant measurements is > 1 day (24 hours)
train_a = remove_constant_intervals(train_a,24)
train_b = remove_constant_intervals(train_b,24)
train_c = remove_constant_intervals(train_c,24)

In [9]:
train_a

,time,pv_measurement
0,2019-06-02 22:00:00,0.00
1,2019-06-02 23:00:00,0.00
2,2019-06-03 00:00:00,0.00
3,2019-06-03 01:00:00,0.00
4,2019-06-03 02:00:00,19.36
...,...,...
34080,2023-04-30 19:00:00,9.02
34081,2023-04-30 20:00:00,0.00
34082,2023-04-30 21:00:00,0.00
34083,2023-04-30 22:00:00,0.00


## 3.Handle missing values and rows 
   - Remove NaN pv measurement values from train a/b/c
   - Remove rows that have timestamps that are not present in both x and y

In [10]:
"""
# Identify the indices of the rows with NaN values in the 'pv_measurement' column
nan_indices_a = train_a[train_a['pv_measurement'].isna()].index
nan_indices_b = train_b[train_b['pv_measurement'].isna()].index
nan_indices_c = train_c[train_c['pv_measurement'].isna()].index

# Drop these indices from y_train
train_a = train_a.drop(nan_indices_a).reset_index(drop = True)
train_b = train_b.drop(nan_indices_b).reset_index(drop = True)
train_c = train_c.drop(nan_indices_c).reset_index(drop = True)

"""

"\n# Identify the indices of the rows with NaN values in the 'pv_measurement' column\nnan_indices_a = train_a[train_a['pv_measurement'].isna()].index\nnan_indices_b = train_b[train_b['pv_measurement'].isna()].index\nnan_indices_c = train_c[train_c['pv_measurement'].isna()].index\n\n# Drop these indices from y_train\ntrain_a = train_a.drop(nan_indices_a).reset_index(drop = True)\ntrain_b = train_b.drop(nan_indices_b).reset_index(drop = True)\ntrain_c = train_c.drop(nan_indices_c).reset_index(drop = True)\n\n"

In [11]:
# Remove all rows with date-time values that are not present in both x and y in order to synchronize x and its labels. 
def remove_non_synchronous_rows(x_train, y_train, x_date_column='date_forecast', y_date_column='time'):
    # Convert date columns to datetime format for easier comparison
    x_train[x_date_column] = pd.to_datetime(x_train[x_date_column])
    y_train[y_date_column] = pd.to_datetime(y_train[y_date_column])
    
    # Find common dates
    common_dates = x_train[x_date_column][x_train[x_date_column].isin(y_train[y_date_column])]
    
    # Filter both datasets based on common dates
    x_train_synced = x_train.loc[x_train[x_date_column].isin(common_dates)]
    y_train_synced = y_train.loc[y_train[y_date_column].isin(common_dates)]
    
    return x_train_synced, y_train_synced

# Remove the rows with date and time that only shows up in one of the sets
x_train_a_hourly, train_a = remove_non_synchronous_rows(x_train_a_hourly, train_a)
x_train_b_hourly, train_b = remove_non_synchronous_rows(x_train_b_hourly, train_b)
x_train_c_hourly, train_c = remove_non_synchronous_rows(x_train_c_hourly, train_c)

# Remove the rows with date and time that only shows up in one of the sets
x_train_a_hourly2, train_a = remove_non_synchronous_rows(x_train_a_hourly2, train_a)
x_train_b_hourly2, train_b = remove_non_synchronous_rows(x_train_b_hourly2, train_b)
x_train_c_hourly2, train_c = remove_non_synchronous_rows(x_train_c_hourly2, train_c)

In [12]:
x_train_a_hourly2

,date_forecast,date_calc,absolute_humidity_2m:gm3_Q0,absolute_humidity_2m:gm3_Q1,absolute_humidity_2m:gm3_Q2,absolute_humidity_2m:gm3_Q3,air_density_2m:kgm3_Q0,air_density_2m:kgm3_Q1,air_density_2m:kgm3_Q2,air_density_2m:kgm3_Q3,ceiling_height_agl:m_Q0,ceiling_height_agl:m_Q1,ceiling_height_agl:m_Q2,ceiling_height_agl:m_Q3,clear_sky_energy_1h:J_Q0,clear_sky_energy_1h:J_Q1,clear_sky_energy_1h:J_Q2,clear_sky_energy_1h:J_Q3,clear_sky_rad:W_Q0,clear_sky_rad:W_Q1,clear_sky_rad:W_Q2,clear_sky_rad:W_Q3,cloud_base_agl:m_Q0,cloud_base_agl:m_Q1,cloud_base_agl:m_Q2,cloud_base_agl:m_Q3,dew_or_rime:idx_Q0,dew_or_rime:idx_Q1,dew_or_rime:idx_Q2,dew_or_rime:idx_Q3,dew_point_2m:K_Q0,dew_point_2m:K_Q1,dew_point_2m:K_Q2,dew_point_2m:K_Q3,diffuse_rad:W_Q0,diffuse_rad:W_Q1,diffuse_rad:W_Q2,diffuse_rad:W_Q3,diffuse_rad_1h:J_Q0,diffuse_rad_1h:J_Q1,diffuse_rad_1h:J_Q2,diffuse_rad_1h:J_Q3,direct_rad:W_Q0,direct_rad:W_Q1,direct_rad:W_Q2,direct_rad:W_Q3,direct_rad_1h:J_Q0,direct_rad_1h:J_Q1,direct_rad_1h:J_Q2,direct_rad_1h:J_Q3,effective_cloud_cover:p_Q0,effective_cloud_cover:p_Q1,effective_cloud_cover:p_Q2,effective_cloud_cover:p_Q3,elevation:m_Q0,elevation:m_Q1,elevation:m_Q2,elevation:m_Q3,fresh_snow_12h:cm_Q0,fresh_snow_12h:cm_Q1,fresh_snow_12h:cm_Q2,fresh_snow_12h:cm_Q3,fresh_snow_1h:cm_Q0,fresh_snow_1h:cm_Q1,fresh_snow_1h:cm_Q2,fresh_snow_1h:cm_Q3,fresh_snow_24h:cm_Q0,fresh_snow_24h:cm_Q1,fresh_snow_24h:cm_Q2,fresh_snow_24h:cm_Q3,fresh_snow_3h:cm_Q0,fresh_snow_3h:cm_Q1,fresh_snow_3h:cm_Q2,fresh_snow_3h:cm_Q3,fresh_snow_6h:cm_Q0,fresh_snow_6h:cm_Q1,fresh_snow_6h:cm_Q2,fresh_snow_6h:cm_Q3,is_day:idx_Q0,is_day:idx_Q1,is_day:idx_Q2,is_day:idx_Q3,is_in_shadow:idx_Q0,is_in_shadow:idx_Q1,is_in_shadow:idx_Q2,is_in_shadow:idx_Q3,msl_pressure:hPa_Q0,msl_pressure:hPa_Q1,msl_pressure:hPa_Q2,msl_pressure:hPa_Q3,precip_5min:mm_Q0,precip_5min:mm_Q1,precip_5min:mm_Q2,precip_5min:mm_Q3,precip_type_5min:idx_Q0,precip_type_5min:idx_Q1,precip_type_5min:idx_Q2,precip_type_5min:idx_Q3,pressure_100m:hPa_Q0,pressure_100m:hPa_Q1,pressure_100m:hPa_Q2,pressure_100m:hPa_Q3,pressure_50m:hPa_Q0,pressure_50m:hPa_Q1,pressure_50m:hPa_Q2,pressure_50m:hPa_Q3,prob_rime:p_Q0,prob_rime:p_Q1,prob_rime:p_Q2,prob_rime:p_Q3,rain_water:kgm2_Q0,rain_water:kgm2_Q1,rain_water:kgm2_Q2,rain_water:kgm2_Q3,relative_humidity_1000hPa:p_Q0,relative_humidity_1000hPa:p_Q1,relative_humidity_1000hPa:p_Q2,relative_humidity_1000hPa:p_Q3,sfc_pressure:hPa_Q0,sfc_pressure:hPa_Q1,sfc_pressure:hPa_Q2,sfc_pressure:hPa_Q3,snow_density:kgm3_Q0,snow_density:kgm3_Q1,snow_density:kgm3_Q2,snow_density:kgm3_Q3,snow_depth:cm_Q0,snow_depth:cm_Q1,snow_depth:cm_Q2,snow_depth:cm_Q3,snow_drift:idx_Q0,snow_drift:idx_Q1,snow_drift:idx_Q2,snow_drift:idx_Q3,snow_melt_10min:mm_Q0,snow_melt_10min:mm_Q1,snow_melt_10min:mm_Q2,snow_melt_10min:mm_Q3,snow_water:kgm2_Q0,snow_water:kgm2_Q1,snow_water:kgm2_Q2,snow_water:kgm2_Q3,sun_azimuth:d_Q0,sun_azimuth:d_Q1,sun_azimuth:d_Q2,sun_azimuth:d_Q3,sun_elevation:d_Q0,sun_elevation:d_Q1,sun_elevation:d_Q2,sun_elevation:d_Q3,super_cooled_liquid_water:kgm2_Q0,super_cooled_liquid_water:kgm2_Q1,super_cooled_liquid_water:kgm2_Q2,super_cooled_liquid_water:kgm2_Q3,t_1000hPa:K_Q0,t_1000hPa:K_Q1,t_1000hPa:K_Q2,t_1000hPa:K_Q3,total_cloud_cover:p_Q0,total_cloud_cover:p_Q1,total_cloud_cover:p_Q2,total_cloud_cover:p_Q3,visibility:m_Q0,visibility:m_Q1,visibility:m_Q2,visibility:m_Q3,wind_speed_10m:ms_Q0,wind_speed_10m:ms_Q1,wind_speed_10m:ms_Q2,wind_speed_10m:ms_Q3,wind_speed_u_10m:ms_Q0,wind_speed_u_10m:ms_Q1,wind_speed_u_10m:ms_Q2,wind_speed_u_10m:ms_Q3,wind_speed_v_10m:ms_Q0,wind_speed_v_10m:ms_Q1,wind_speed_v_10m:ms_Q2,wind_speed_v_10m:ms_Q3,wind_speed_w_1000hPa:ms_Q0,wind_speed_w_1000hPa:ms_Q1,wind_speed_w_1000hPa:ms_Q2,wind_speed_w_1000hPa:ms_Q3
0,2019-06-02 22:00:00,NaT,7.7,7.7,7.7,7.7,1.230,1.229,1.228,1.226,1744.900024,1734.000000,1723.500000,1713.400024,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1744.900024,1734.000000,1723.500000,1713.400024,0.0,0.0,0.0,0.0,280.299988,280.2999